In [1]:
import math
from ase import Atoms
from ase.visualize import view
from ase.build import nanotube

import nglview as nv

import IPython

In [2]:
# (n, m) = (5, 5)
cnt = nanotube(5, 5, length=1)

cnt = nanotube(5, 5, length=4, bond=1.42)
w = nv.show_ase(cnt)
w.add_ball_and_stick()
w.add_label(color="black", labelType="atomindex")

IPython.display.display(w)

NGLWidget()

In [3]:
from ase.io import write
write("cnt.xyz", cnt) #запить структуры в файл .xyz

In [4]:
import numpy as np
from ase.neighborlist import NeighborList

def print_neighbors_with_coords(cnt, cutoff=1.43):
    """
    Для каждого атома выводит:
      - индекс
      - координаты
      - индексы ближайших соседей
      - координаты соседей с учетом PBC (offset)
      - расстояния
    """
    radii = [cutoff / 2.0] * len(cnt)   #передаем радиусы, в пределах которрых ищем соседей

    nl = NeighborList(
        radii,
        self_interaction=False,
        bothways=True
    )
    nl.update(cnt)

    cell = cnt.cell.array
    pos = cnt.positions

    for i in range(len(cnt)):
        ri = pos[i]
        neigh_idx, offsets = nl.get_neighbors(i)

        print(f"\nAtom {i}")
        print(f"  position: {ri}")

        for j, off in zip(neigh_idx, offsets):
            # координата соседа с учетом периодического сдвига
            rj = pos[j] + off @ cell
            d = np.linalg.norm(rj - ri)

            print(
                f"    neighbor {j:3d} | offset {tuple(off)} | "
                f"pos {rj} | dist {d:.3f} Å"
            )

print_neighbors_with_coords(cnt)



Atom 0
  position: [3.39000029 0.         0.        ]
    neighbor   1 | offset (np.int64(0), np.int64(0), np.int64(0)) | pos [3.09691936 1.37883734 0.        ] | dist 1.410 Å
    neighbor  19 | offset (np.int64(0), np.int64(0), np.int64(0)) | pos [ 3.31592065 -0.70482069  1.22975607] | dist 1.419 Å
    neighbor  79 | offset (np.int64(0), np.int64(0), np.int64(-1)) | pos [ 3.31592065 -0.70482069 -1.22975607] | dist 1.419 Å

Atom 1
  position: [3.09691936 1.37883734 0.        ]
    neighbor   2 | offset (np.int64(0), np.int64(0), np.int64(0)) | pos [2.74256784 1.99259217 1.22975607] | dist 1.419 Å
    neighbor   0 | offset (np.int64(0), np.int64(0), np.int64(0)) | pos [3.39000029 0.         0.        ] | dist 1.410 Å
    neighbor  62 | offset (np.int64(0), np.int64(0), np.int64(-1)) | pos [ 2.74256784  1.99259217 -1.22975607] | dist 1.419 Å

Atom 2
  position: [2.74256784 1.99259217 1.22975607]
    neighbor   3 | offset (np.int64(0), np.int64(0), np.int64(0)) | pos [1.69500014 2.935826

In [10]:
import numpy as np
from ase.neighborlist import NeighborList

def get_neighbor_pair_coords(cnt, i, j, cutoff=1.43):
    """
    Проверяет, являются ли атомы i и j ближайшими соседями (с учётом PBC)

    Если да:
        возвращает (ri, rj_real)
        ri       — координаты атома i
        rj_real  — координаты атома j с учетом периодического сдвига
    Если нет:
        возвращает 0
    """
    radii = [cutoff / 2.0] * len(cnt)

    nl = NeighborList(
        radii,
        self_interaction=False,
        bothways=True
    )
    nl.update(cnt)

    # Получаем соседей атома i
    neigh_idx, offsets = nl.get_neighbors(i)

    cell = cnt.cell.array
    pos = cnt.positions

    for k, off in zip(neigh_idx, offsets):
        if k == j:
            ri = pos[i]
            rj_real = pos[j] + off @ cell
            return ri, rj_real

    return 0

i, j = 0, 1

res = get_neighbor_pair_coords(cnt, i, j)
print(res[1])

if res == 0:
    print(f"Атомы {i} и {j} НЕ соседи")
else:
    ri, rj = res
    print(f"Атомы {i} и {j} — соседи")
    print("ri =", ri)
    print("rj =", rj)
    print("distance =", np.linalg.norm(rj - ri))


[3.09691936 1.37883734 0.        ]
Атомы 0 и 1 — соседи
ri = [3.39000029 0.         0.        ]
rj = [3.09691936 1.37883734 0.        ]
distance = 1.4096413834422334
